In [1]:
import os
import boto3
import pyaudio
import irc.client
import irc.events
polly = boto3.client('polly')
p = pyaudio.PyAudio()

def chat_handler(connection, event):
    if 'randallatamazon' in event.source:
        voice = 'Brian'
    else:
        voice = 'Justin'
    text = event.arguments[0]
#     sentiment = comprehend.detect_sentiment(Text=text, LanguageCode='en')['Sentiment']
#     if sentiment == "NEGATIVE":
#         print "NEGATIVE SENTIMENT!"
#         return
    resp = polly.synthesize_speech(Text=text, TextType="text", OutputFormat="pcm", VoiceId=voice)
    stream = p.open(format=p.get_format_from_width(width=2), channels=1, rate=16000, output=True)
    stream.write(resp['AudioStream'].read())
    stream.stop_stream()
    stream.close()

client = irc.client.Reactor()
client.add_global_handler('pubmsg', chat_handler)
server = client.server()
server.connect("irc.chat.twitch.tv", 6667, "justinfan123")
server.join('#aws')
print server.is_connected()
client.process_forever()

True


KeyboardInterrupt: 

In [ ]:
import os
os.getenv('twitch_pass')

In [ ]:
help(irc.events.all)

In [ ]:
    buf = StringIO.StringIO()


In [ ]:
comprehend = boto3.client('comprehend')
help(comprehend.detect_sentiment)
sentiment = comprehend.detect_sentiment(Text=text, LanguageCode='en')['Sentiment']
if sentiment == "NEGATIVE":
    print "NEGATIVE SENTIMENT!"
    continue

In [ ]:
import os
import boto3
import pyaudio
import irc.client
import irc.events
from babylex import LexSession
polly = boto3.client('polly')
p = pyaudio.PyAudio()

sessions = dict()

def chat_handler(connection, event):
    message = event.arguments[0].split(":", 1)
    if len(message) > 1 and message[0].lower() == connection.get_nickname().lower():
        user = event.source.split('!')[0]
        if user not in sessions:
            sessions[user] = LexSession("TwitchChatBot", "$LATEST", user)
        session = sessions[user]
        resp = session.content(message[1], "text/plain; charset=utf-8", "audio/pcm")
        stream = p.open(format=p.get_format_from_width(width=2), channels=1, rate=16000, output=True)
        stream.write(resp.content)
        stream.stop_stream()
        stream.close()
        connection.privmsg(event.target, resp.headers['x-amz-lex-message'])

client = irc.client.Reactor()
client.add_global_handler('pubmsg', chat_handler)
server = client.server()
server.connect("irc.chat.twitch.tv", 6667, "awslexbot", password=os.getenv('twitch_pass'))
server.join('#aws')
print server.is_connected()
server.privmsg('#aws', "HELLO, WORLD!")
client.process_forever()